In [1]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [2]:
connection = sqlite3.connect('../../data/checking-logs.sqlite')

## get the schema of the table test

In [3]:
pd.io.sql.read_sql("PRAGMA table_info(test);", connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test to check what the table looks like

In [4]:
pd.io.sql.read_sql("SELECT * FROM test LIMIT 10;", connection)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query
  * do this by joining the table with the table deadlines
  * the difference should be displayed in hours
  * do not take the lab ’project1’ into account, it has longer deadlines and will be
  an outlier
  * the value should be stored in the dataframe df_min with the corresponding
  uid

In [5]:
df_min = pd.io.sql.read_sql("SELECT uid, MIN(diff) FROM "
"(SELECT t.uid, t.labname, t.first_commit_ts, CAST((- julianday(datetime(d.deadlines, 'unixepoch')) + julianday(t.first_commit_ts))*24 AS INTEGER) AS diff FROM test as t " \
"JOIN deadlines as d " \
"ON d.labs = t.labname " \
"WHERE t.labname != 'project1')", connection)
df_min

,uid,MIN(diff)
0,user_30,-202


## do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [6]:
df_max = pd.io.sql.read_sql("SELECT uid, MAX(diff) FROM "
"(SELECT t.uid, t.labname, t.first_commit_ts, CAST((-julianday(datetime(d.deadlines, 'unixepoch')) + julianday(t.first_commit_ts))*24 AS INTEGER) AS diff FROM test as t " \
"JOIN deadlines as d " \
"ON d.labs = t.labname " \
"WHERE t.labname != 'project1')", connection)
df_max

,uid,MAX(diff)
0,user_25,-2


## do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [7]:
df_avg = pd.io.sql.read_sql("SELECT AVG(diff) FROM "
"(SELECT t.uid, t.labname, t.first_commit_ts, CAST((-julianday(datetime(d.deadlines, 'unixepoch')) + julianday(t.first_commit_ts))*24 AS INTEGER) AS diff FROM test as t " \
"JOIN deadlines as d " \
"ON d.labs = t.labname " \
"WHERE t.labname != 'project1')", connection)
df_avg

,AVG(diff)
0,-89.125


## we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference
  * using only one query, create a table with the columns: uid, avg_diff, pageviews
  * uid is the uids that exist in the test
  * avg_diff is the average delta between the first commit and the lab deadline per user
  * pageviews is the number of Newsfeed visits per user
  * do not take the lab ’project1’ into account
  * store it to the dataframe views_diff
  * use the Pandas method corr() to calculate the correlation coefficient between
the number of pageviews and the difference

In [8]:
views_diff = pd.io.sql.read_sql("SELECT t.uid, t.diff, p.pageviews FROM "
"(SELECT t.uid, t.labname, t.first_commit_ts, CAST((-julianday(datetime(d.deadlines, 'unixepoch')) + julianday(t.first_commit_ts))*24 AS INTEGER) AS diff FROM test as t " \
"JOIN deadlines as d " \
"ON d.labs = t.labname " \
"WHERE t.labname != 'project1') as t " \
"JOIN " \
"(SELECT uid, COUNT(*) as pageviews FROM pageviews " \
"GROUP BY uid) " \
"as p ON p.uid = t.uid  "  \
"GROUP BY t.uid", connection)
views_diff

,uid,diff,pageviews
0,user_1,-6,28
1,user_10,-39,89
2,user_14,-200,143
3,user_17,-81,47
4,user_18,-4,3
5,user_19,-148,16
6,user_21,-126,10
7,user_25,-148,179
8,user_28,-98,149
9,user_3,-75,317


In [9]:
views_diff.drop(columns=['uid']).corr()

,diff,pageviews
diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


## close the connection

In [114]:
connection.close()